- now, we're going to program Extended Kalman filter
- it's actually hard to do it turns out because even small errors in computer derivatives lead to completely bizarre behaviour in the output and they're very hard to debug

# Intro to Extended Kalman Filter Project

- now that you have learned how the Extended Kalman filter works, you are going to implement the Extended Kalman filter in C++
- we are providing simulated lidar and radar measurements detecting a bicycle that travels around your vehicle
- you will use a Kalman filter, lidar measurements and radar measurements to track the bicycle's position and velocity


- lidar measurements are red circles, radar measurements are blue circles with an arrow pointing in the direction of the observed angle, and estimation markers are green triangles
- the video below shows what the simulator looks like when a C++ script is using its Kalman filter to track the object
- the simulator provides the script the measured data (either lidar or radar), and the script feeds back the measured estimation marker, and RMSE values from its Kalman filter
- https://youtu.be/d6qbR3_LPoA

## Example of Tracking with Lidar

- check out the video below to see a real world example of object tracking with lidar
- in this project, you will only be tracking one object, but the video will give you a sense for how object tracking with lidar works
- https://youtu.be/FMNJPX_sszU

# Data File for EKF project

- the GitHub repo contains one data file: `obj_pose-laser-radar-synthetic-input.txt`
- the simulator will be using this data file, and feed `main.cpp` values from it one line at a time
- here is a screenshot of the first data file:

<img src="resources/data_file_screenshot.png"/>

- each row represents a sensor measurement where the first column tells you if the measurement comes from radar (R) or lidar (L)


- for a row containing `radar data`, the columns are: `sensor_type`, `rho_measured`, `phi_measured`, `rhodot_measured`, `timestamp`, `x_groundtruth`, `y_groundtruth`, `vx_groundtruth`, `vy_groundtruth`, `yaw_groundtruth`, `yawrate_groundtruth`


- for a row containing `lidar data`, the columns are: `sensor_type`, `x_measured`, `y_measured`, `timestamp`, `x_groundtruth`, `y_groundtruth`, `vx_groundtruth`, `vy_groundtruth`, `yaw_groundtruth`, `yawrate_groundtruth`


- whereas radar has three measurements (rho, phi, rhodot), lidar has two measurements (x, y)
- you will use the measurement values and timestamp in your Kalman filter algorithm
  - groundtruth, which represents the actual path the bicycle took, is for calculating root mean squared error
  - you do not need to worry about yaw and yaw rate ground truth values

### Reading in the Data

- we have provided code that will read in and parse the data files for you
- this code is in the `main.cpp` file
  - the `main.cpp` file creates instances of a `MeasurementPackage`


- if you look inside `main.cpp`, you will see code like:
```cpp
MeasurementPackage meas_package;
meas_package.sensor_type_ = MeasurementPackage::LASER;
meas_package.raw_measurements_ = VectorXd(2);
meas_package.raw_measurements_ << px, py;
meas_package.timestamp_ = timestamp;
```
and
```cpp
vector<VectorXd> ground_truth;
VectorXd gt_values(4);
gt_values(0) = x_gt;
gt_values(1) = y_gt; 
gt_values(2) = vx_gt;
gt_values(3) = vy_gt;
ground_truth.push_back(gt_values);
```

- the code reads in the data file line by line
- the measurement data for each line gets pushed onto a `measurement_pack_list`
- the ground truth $[p_x, p_y, v_x, v_y]$ for each line in the data file gets pushed onto `ground_truth` so RMSE can be calculated later from `tools.cpp`

# File Structure

## Overview of a Kalman Filter: Initialize, Predict, Update

- to review what we learned in the extended Kalman filter lectures, let's discuss the three main steps for programming a Kalman filter:
  - **initializing** Kalman filter variables
  - **predicting** where our object is going to be after a time step $\Delta{t}$
  - **updating** where our object is based on sensor measurements


- then the prediction and update steps repeat themselves in a loop


- to measure how well our Kalman filter performs, we will then calculate **root mean squared error** comparing the Kalman filter results with the provided ground truth


- these three steps (initialize, predict, update) plus calculating RMSE encapsulate the entire extended Kalman filter project

## Files in the GitHub src folder

- the files you need to work with are in the `src` folder of the GitHub repository
  - `main.cpp` - communicates with the Term 2 Simulator receiving data measurements, calls a function to run the Kalman filter, calls a function to calculate RMSE
  - `FusionEKF.cpp` - initializes the filter, calls the predict function, calls the update function
  - `kalman_filter.cpp` - defines the predict function, the update function for lidar, and the update function for radar
  - `tools.cpp` - function to calculate RMSE and the Jacobian matrix


- the only files you need to modify are `FusionEKF.cpp`, `kalman_filter.cpp`, and `tools.cpp`

## How the Files Relate to Each Other

- here is a brief overview of what happens when you run the code files:
  1. `main.cpp` reads in the data and sends a sensor measurement to `FusionEKF.cpp`
  2. `FusionEKF.cpp` takes the sensor data and initializes variables and updates variables
    - the Kalman filter equations are not in this file
    - `FusionEKF.cpp` has a variable called `ekf_`, which is an instance of a `KalmanFilter` class
    - the `ekf_` will hold the matrix and vector values
    - you will also use the `ekf_` instance to call the predict and update equations
  3. the `KalmanFilter` class is defined in `kalman_filter.cpp` and `kalman_filter.h`
    - you will only need to modify `kalman_filter.cpp`, which contains functions for the prediction and update steps

## Main.cpp

- although you will not need to modify this file, the project is easier to implement once you understand what the file is doing
- as a suggestion, open the GitHub repository for the project and look at the code files simultaneously with this lecture slide


- the Term 2 simulator is a client, and the C++ program software is a web server


- we already discussed how `main.cpp` reads in the sensor data
- recall that `main.cpp` reads in the sensor data line by line from the client and stores the data into a measurement object that it passes to the Kalman filter for processing
- also a ground truth list and an estimation list are used for tracking RMSE


- `main.cpp` is made up of several functions within `main()`, these all handle the uWebsocketIO communication between the simulator and it's self


- here is the main protocol that main.cpp uses for uWebSocketIO in communicating with the simulator:

```cpp
INPUT: values provided by the simulator to the C++ program

["sensor_measurement"] => the measurement that the simulator observed (either lidar or radar)


OUTPUT: values provided by the C++ program to the simulator

["estimate_x"] <= kalman filter estimated position x
["estimate_y"] <= kalman filter estimated position y
["rmse_x"]
["rmse_y"]
["rmse_vx"]
["rmse_vy"]
```

- all the main code loops in `h.onMessage()`, to have access to intial variables that we created at the beginning of `main()`, we pass pointers as arguments into the header of `h.onMessage()`
- for example
```cpp
h.onMessage([&fusionEKF,&tools,&estimations,&ground_truth]
            (uWS::WebSocket<uWS::SERVER> ws, char *data, size_t length, 
             uWS::OpCode opCode)
```

- the rest of the arguments in `h.onMessage` are used to set up the server
```cpp
 // Create a Fusion EKF instance
  FusionEKF fusionEKF;

  // used to compute the RMSE later
  vector<VectorXd> estimations;
  vector<VectorXd> ground_truth;

  //Call the EKF-based fusion
  fusionEKF.ProcessMeasurement(meas_package); 
```
- the code is:
  - creating an instance of the `FusionEKF` class
  - receiving the measurement data calling the `ProcessMeasurement()` function
    - `ProcessMeasurement()` is responsible for the initialization of the Kalman filter as well as calling the prediction and update steps of the Kalman filter
    - you will be implementing the `ProcessMeasurement()` function in `FusionEKF.cpp`


- the rest of `main.cpp` will output the following results to the simulator:
  - estimation position
  - calculated RMSE
- `main.cpp` will call a function to calculate root mean squared error:
```cpp
  // compute the accuracy (RMSE)
  Tools tools;
  cout << "Accuracy - RMSE:" << endl << tools.CalculateRMSE(estimations, ground_truth) << endl;
```
  - you will implement an RMSE function in the `tools.cpp` file

# Project Code

## FusionEKF.cpp

- in `FusionEKF.cpp`, we have given some starter code for implementing sensor fusion
  - in this file, you won't need to include the actual Kalman filter equations; instead, you will be initializing variables, initializing the Kalman filters, and then calling functions that implement the prediction step or update step
  - you will see TODO comments indicating where to put your code


- you will need to:
  - initialize variables and matrices (x, F, H_laser, H_jacobian, P, etc.)
  - initialize the Kalman filter position vector with the first sensor measurements
  - modify the F and Q matrices prior to the prediction step based on the elapsed time between measurements
  - call the update step for either the lidar or radar sensor measurement. Because the update step for lidar and radar are slightly different, there are different functions for updating lidar and radar

### Initializing Variables in FusionEKF.cpp

```cpp
// initializing matrices
  R_laser_ = MatrixXd(2, 2);
  R_radar_ = MatrixXd(3, 3);
  H_laser_ = MatrixXd(2, 4);
  Hj_ = MatrixXd(3, 4);

  //measurement covariance matrix - laser
  R_laser_ << 0.0225, 0,
              0, 0.0225;

  //measurement covariance matrix - radar
  R_radar_ << 0.09, 0, 0,
              0, 0.0009, 0,
              0, 0, 0.09;

  /**
   * TODO: Finish initializing the FusionEKF.
   * TODO: Set the process and measurement noises
   */
```

- every time `main.cpp` calls `fusionEKF.ProcessMeasurement(measurement_pack_list[k])`, the code in `FusionEKF.cpp` will run
  - if this is the first measurement, the Kalman filter will try to initialize the object's location with the sensor measurement

### Initializing the Kalman Filter in FusionEKF.cpp

```cpp
/**
   * Initialization
   */
  if (!is_initialized_) {
    /**
     * TODO: Initialize the state ekf_.x_ with the first measurement.
     * TODO: Create the covariance matrix.
     * You'll need to convert radar from polar to cartesian coordinates.
     */

    // first measurement
    cout << "EKF: " << endl;
    ekf_.x_ = VectorXd(4);
    ekf_.x_ << 1, 1, 1, 1;

    if (measurement_pack.sensor_type_ == MeasurementPackage::RADAR) {
      // TODO: Convert radar from polar to cartesian coordinates 
      //         and initialize state.

    }
    else if (measurement_pack.sensor_type_ == MeasurementPackage::LASER) {
      // TODO: Initialize state.

    }

    // done initializing, no need to predict or update
    is_initialized_ = true;
    return;
  }
```

### Predict and Update Steps in FusionEKF.cpp

- once the Kalman filter gets initialized, the next iterations of the for loop will call the `ProcessMeasurement()` function to do the predict and update steps

```cpp
/**
   * Prediction
   */

  /**
   * TODO: Update the state transition matrix F according to the new elapsed time.
   * Time is measured in seconds.
   * TODO: Update the process noise covariance matrix.
   * Use noise_ax = 9 and noise_ay = 9 for your Q matrix.
   */

  ekf_.Predict();

  /**
   * Update
   */

  /**
   * TODO: Use the sensor type to perform the update step.
   * TODO: Update the state and covariance matrices.
   */

  if (measurement_pack.sensor_type_ == MeasurementPackage::RADAR) {
    // TODO: Radar updates

  } else {
    // TODO: Laser updates

  }
```

- in `FusionEKF.cpp`, you will see references to a variable called `ekf_`
  - the `ekf_` variable is an instance of the `KalmanFilter` class
  - you will use `ekf_` to store your Kalman filter variables (x, P, F, H, R, Q) and call the predict and update functions

## KalmanFilter Class

- `kalman_filter.h` defines the `KalmanFilter` class containing the x vector as well as the P, F, Q, H and R matrices
- the KalmanFilter class also contains functions for the prediction step as well as the Kalman filter update step (lidar) and extended Kalman filter update step (radar).


- you will need to add your code to `kalman_filter.cpp` to implement the prediction and update equations
- you do not need to modify `kalman_filter.h`


- because lidar uses linear equations, the update step will use the basic Kalman filter equations
- on the other hand, radar uses non-linear equations, so the update step involves linearizing the equations with the Jacobian matrix
- the `Update` function will use the standard Kalman filter equations
- the `UpdateEKF` will use the extended Kalman filter equations:

```cpp
void KalmanFilter::Predict() {
  /**
   * TODO: predict the state
   */
}

void KalmanFilter::Update(const VectorXd &z) {
  /**
   * TODO: update the state by using Kalman Filter equations
   */
}

void KalmanFilter::UpdateEKF(const VectorXd &z) {
  /**
   * TODO: update the state by using Extended Kalman Filter equations
   */
}
```

## Tools.cpp

- this file is relatively straight forward--you will implement functions to calculate root mean squared error and the Jacobian matrix:

```cpp
VectorXd Tools::CalculateRMSE(const vector<VectorXd> &estimations,
                              const vector<VectorXd> &ground_truth) {
  /**
   * TODO: Calculate the RMSE here.
   */
}

MatrixXd Tools::CalculateJacobian(const VectorXd& x_state) {
  /**
   * TODO: Calculate a Jacobian here.
   */
}
```

# Tips and Tricks

### Summary of What Needs to Be Done

- in `tools.cpp`, fill in the functions that calculate root mean squared error (RMSE) and the Jacobian matrix
- fill in the code in `FusionEKF.cpp`
  - you'll need to initialize the Kalman Filter, prepare the Q and F matrices for the prediction step, and call the radar and lidar update functions
- in `kalman_filter.cpp`, fill out the `Predict()`, `Update()`, and `UpdateEKF()` functions

### Tips and Tricks

- Review the Previous Lessons
  - review the previous lessons! Andrei, Dominik and co. have given you everything you need
  - in fact, you've built most of an Extended Kalman Filter already!
  - take a look at the programming assignments and apply the techniques you used to this project


- No Need to Tune Parameters
  - the R matrix values and Q noise values are provided for you
  - there is no need to tune these parameters for this project


- Initializing the State Vector
  - you'll need to initialize the state vector with the first sensor measurement
  - although radar gives velocity data in the form of the range rate $\dot{\rho}$, a radar measurement does not contain enough information to determine the state variable velocities $v_x$ and $v_y$
    - you can, however, use the radar measurements $\rho$ and $\phi$ to initialize the state variable locations $p_x$ and $p_y$


- Calculating $y = z - H * x'$
  - for lidar measurements, the error equation is $y = z - H * x'$
  - for radar measurements, the functions that map the $x$ vector $[p_x, p_y, v_x, v_y]$ to polar coordinates are non-linear
    - instead of using $H$ to calculate $y = z - H * x'$, for radar measurements you'll have to use the equations that map from cartesian to polar coordinates: $y = z - h(x')$


- Normalizing Angles
  - in C++, `atan2()` returns values between $-\pi$ and $\pi$
  - when calculating $\phi$ in $y = z - h(x)$ for radar measurements, the resulting angle $\phi$ in the $y$ vector should be adjusted so that it is between $-\pi$ and $\pi$
  - the Kalman filter is expecting small angle values between the range $-\pi$ and $\pi$
  - HINT: when working in radians, you can add $2\pi$ or subtract $2\pi$ until the angle is within the desired range


- Avoid Divide by Zero throughout the Implementation
  - before and while calculating the Jacobian matrix Hj, make sure your code avoids dividing by zero
  - for example, both the x and y values might be zero or `px*px + py*py` might be close to zero


- Test Your Implementation
  - test! We're giving you the ability to analyze your output data and calculate RMSE
  - as you make changes, keep testing your algorithm!
  - if you are getting stuck, add print statements to pinpoint any issues--but please remove extra print statements before turning in the code

### Ideas for Standing out!

- the Kalman Filter general processing flow that you've learned in the preceding lessons gives you the basic knowledge needed to track an object
- however, there are ways that you can make your algorithm more efficient!
  - dealing with the first frame, in particular, offers opportunities for improvement
  - experiment and see how low your RMSE can go!
  - try removing radar or lidar data from the filter
    - observe how your estimations change when running against a single sensor type!--do the results make sense given what you know about the nature of radar and lidar data?
  - we give you starter code, but you are not required to use it!
    - you may want to start from scratch

### Optional Resources

- to complete the project, you only need the files in the GitHub repo; however, we are also providing some extra resources that you can use to develop your solution:
  - a [Sensor Fusion utilities repo](https://github.com/udacity/CarND-Mercedes-SF-Utilities) containing Matlab scripts that will generate more sample data (generating your own sample data is completely optional)
  - visualization package that you can also find within the [Sensor Fusion utilities repo](https://github.com/udacity/CarND-Mercedes-SF-Utilities)

# Project Instructions for local setup

### Download Link for EKF Simulator

- the EKF project was previously part of Term 2, so please download the [Term 2 Simulator Release](https://github.com/udacity/self-driving-car-sim/releases/)

### Running the Program

- download the simulator and open it
- in the main menu screen select Project 1/2 EKF and UKF
  - once the scene is loaded you can hit the START button to observe how the object moves and how measurement markers are positioned in the data set
  - also for more experimentation, "Data set 2" is included which is a reversed version of "Data set 1", also the second data set starts with a radar measurement where the first data set starts with a lidar measurement
  - at any time you can press the PAUSE button, to pause the scene or hit the RESTART button to reset the scene
  - also, the ARROW KEYS can be used to move the camera around, and the top left ZOOM IN/OUT buttons can be used to focus the camera
  - pressing the ESCAPE KEY returns to the simulator main menu
- the EKF project GitHub repository README has more detailed instructions for installing and using C++ uWebScoketIO.


### NOTES:

- currently hitting Restart or switching between data sets only refreshes the simulator state and not the Kalman Filter's saved results
  - the current procedure for refreshing the Kalman Filter is to close the connection, ctrl+c and reopen it, ./ExtendedKF
  - if you don't do this when trying to run a different data set or running the same data set multiple times in a row, the RMSE values will become large because of the previous different filter results still being observed in memory
- the two functions in `tools.cpp` need to be implemented for the simulator to avoid throwing a segmentation fault - make sure to get those built out in order to test in the simulator!
- students have reported rapid expansion of log files when using the term 2 simulator
  - this appears to be associated with not being connected to uWebSockets
  - if this does occur, please make sure you are connected to uWebSockets
  - the following workaround may also be effective at preventing large log files
    - create an empty log file
    - remove write permissions so that the simulator can't write to log

# uWebSocketIO Starter Guide

- this project involve using an open source package called [uWebSocketIO](https://github.com/uNetworking/uWebSockets)
- this package facilitates the same connection between the simulator and code that was used in the Term 1 Behavioral Cloning Project, but now with C++
- the package does this by setting up a web socket server connection from the C++ program to the simulator, which acts as the host
- in the project repository there are two scripts for installing uWebSocketIO - one for Linux and the other for macOS
- note: Only uWebSocketIO branch e94b6e1, which the scripts reference, is compatible with the package installation.


- Linux Installation: from the project repository directory run the script: `install-ubuntu.sh`

### Port forwarding is required when running code on VM and simulator on host

- for security reasons, the VM does not automatically open port forwarding, so you need to manually [enable port 4567](https://www.howtogeek.com/122641/how-to-forward-ports-to-a-virtual-machine-and-use-it-as-a-server/)
  - this is needed for the C++ program to successfully connect to the host simulator
  
- Port Forwarding Instructions
  - first open up Oracle VM VirtualBox
  - click on the default session and select settings
  - click on Network, and then Advanced
  - click on Port Forwarding
  - click on the green plus, adds new port forwarding rule
  - add a rule that assigns 4567 as both the host port and guest Port, as in the screenshot
    
<img src="resources/port_forward.png"/>